In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.layers import Dens, Dropout, Flatten, Conv2D, MaxPooling2D

In [2]:
data = pd.read_csv("data/lerped-data.csv")

positions = data[["x", "y", "z"]].values.reshape(-1, 15, 3)
is_exploiting = data["is_exploiting"].values[::15]

print(positions.shape, is_exploiting.shape)
print(positions, is_exploiting)

(1400, 15, 3) (1400,)
[[[ 2.88        6.78       43.95      ]
  [17.91428571 13.25142857 40.71857143]
  [27.83428571  7.36571429 42.35142857]
  ...
  [ 7.65714286  7.08142857 -2.51142857]
  [15.27571429  5.27571429  0.28285714]
  [61.93        8.67       10.02      ]]

 [[61.72       15.74       32.18      ]
  [12.24571429  8.06857143 39.76571429]
  [14.05       10.59714286 38.85857143]
  ...
  [ 8.05142857  7.37142857  0.23714286]
  [15.66142857  6.17428571  1.66857143]
  [59.35        9.62       10.        ]]

 [[ 4.24        3.64       41.44      ]
  [16.84857143 10.10285714 38.83428571]
  [26.75        7.07285714 41.97142857]
  ...
  [ 6.90428571  6.58714286 -2.28      ]
  [15.36571429  9.04714286 -0.42      ]
  [59.32        8.85       10.74      ]]

 ...

 [[ 2.62        9.08       45.28      ]
  [ 3.76        8.08       31.40571429]
  [ 4.9         7.08       17.53142857]
  ...
  [ 8.58428571  5.21428571 -0.83142857]
  [ 9.02714286  4.69714286 -0.84571429]
  [ 9.47        4.18  

In [3]:
split = int(len(positions) * 0.8)

shuffle_index = np.random.permutation(len(positions))
positions, is_exploiting = positions[shuffle_index], is_exploiting[shuffle_index]

positions_train, positions_test = positions[:split], positions[split:]
is_exploiting_train, is_exploiting_test = is_exploiting[:split], is_exploiting[split:]

print(positions_train.shape, positions_test.shape, is_exploiting_train.shape, is_exploiting_test.shape)

(1120, 15, 3) (280, 15, 3) (1120,) (280,)


In [18]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(15, 3)),
    keras.layers.Dense(64),
    keras.layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer="adam",
                loss="binary_crossentropy",
                metrics=["accuracy"])

model.fit(positions_train, is_exploiting_train, epochs=25, batch_size=50)

Epoch 1/10
75/75 [==============================] - 1s 4ms/step - loss: 0.6388 - accuracy: 0.7732
Epoch 2/10
75/75 [==============================] - 0s 3ms/step - loss: 0.2070 - accuracy: 0.9268
Epoch 3/10
75/75 [==============================] - 0s 3ms/step - loss: 0.1339 - accuracy: 0.9536
Epoch 4/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0983 - accuracy: 0.9741
Epoch 5/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0745 - accuracy: 0.9857
Epoch 6/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0576 - accuracy: 0.9893
Epoch 7/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0492 - accuracy: 0.9893
Epoch 8/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0428 - accuracy: 0.9911
Epoch 9/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0345 - accuracy: 0.9937
Epoch 10/10
75/75 [==============================] - 0s 3ms/step - loss: 0.0283 - accuracy: 0.9937


In [19]:
model.evaluate(positions_test, is_exploiting_test)

9/9 [==============================] - 0s 2ms/step - loss: 0.0462 - accuracy: 0.9857


[0.046161800622940063, 0.9857142567634583]

In [20]:
model.save("models/TA98_7_28_23.h5")